In [4]:
pip install tensorflow

In [18]:
# Import Libraries
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
from keras import layers

import tensorflow_hub as hub
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
# load model
# model_load = keras.models.load_model("FINAL_TRF.tf")

In [6]:
model = tf.keras.models.load_model(
       ('/content/FINAL_TRF.h5'),
       custom_objects={'KerasLayer':hub.KerasLayer}
)

In [3]:
# model=load_model("FINAL_TRF.h5")

ValueError: Unknown layer: 'KerasLayer'. Please ensure you are using a `keras.utils.custom_object_scope` and that this object is included in the scope. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.

In [30]:
#inser teks
message = ['The burger is delicious, I like it!', 'I like the ambiance!', 'The food was tasteless and bland', 'love this place!!']

# Create Dataframe
teks = pd.DataFrame()
teks['message'] = message
teks.head()

,message
0,"The burger is delicious, I like it!"
1,I like the ambiance!
2,The food was tasteless and bland
3,love this place!!


In [31]:
# Create Lemmatizer and Stopwords
stpwds_eng = list(set(stopwords.words('english')))
stpwds_eng.append(['oh','s'])

# lemmatization
nltk.download('wordnet')
# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [32]:
#Create Preprocessing Function
def pre_process(text):

  text = text.lower() #to lowercase

  text = re.sub("@[A-Za-z0-9_]+", " ", text)# Remove Mention

  text = re.sub("#[A-Za-z0-9_]+", " ", text)# Remove Hashtag

  text = re.sub(r"\\n", " ",text)# Remove \n

  text = text.strip() # Remove Whitespace

  text = re.sub(r"http\S+", " ", text) # Remove Link
  text = re.sub(r"www.\S+", " ", text)

  text = re.sub("[^A-Za-z\s']", " ", text)# Remove symbols, emojis

  text = re.sub(r'\b\w{2}\b', '', text)# Remove words with 2 letters or less

  tokens = word_tokenize(text)# Tokenization

  text = ' '.join([word for word in tokens if word not in stpwds_eng])# Remove Stopwords

  text = lemmatizer.lemmatize(text)# Lemmatizing using WordLemmatizer

  return text

In [33]:
## Preprocessing
teks['text_processed'] = teks['message'].apply(lambda x: pre_process(x))
teks

,message,text_processed
0,"The burger is delicious, I like it!",burger delicious like
1,I like the ambiance!,like ambiance
2,The food was tasteless and bland,food tasteless bland
3,love this place!!,love place


In [34]:
# Prediction
teks_prediction = model.predict(teks['text_processed'])
teks_prediction = np.where(teks_prediction>= 0.5,1,0)

predicted_sentiment=[]

for i in teks_prediction:
    if i == 0:
        predicted_sentiment.append('not recommended')
    else :
        predicted_sentiment.append('recommended')

teks['prediction_sentiment'] = predicted_sentiment
teks

1/1 [==============================] - 0s 21ms/step


,message,text_processed,prediction_sentiment
0,"The burger is delicious, I like it!",burger delicious like,recommended
1,I like the ambiance!,like ambiance,recommended
2,The food was tasteless and bland,food tasteless bland,not recommended
3,love this place!!,love place,recommended
